In [1]:
# Importando bibliotecas
from functions import *
import pandas as pd
import locale
from pathlib import Path
import shutil
from datetime import datetime
import warnings
import logging
from openpyxl import load_workbook

timer = Temporizador()
timer.iniciar()

locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.expand_frame_repr', False)

# Detecta se o script está sendo executado de um .py ou de um notebook
try:
    caminho_base = Path(__file__).resolve().parent
except NameError:
    # __file__ não existe em Jupyter ou ambiente interativo
    caminho_base = Path.cwd()


pasta_input_parquet = caminho_base.parent / '01_INPUT_PIPELINE/01_BD_PARQUET'
pasta_staging_parquet = caminho_base.parent / '02_STAGING_PARQUET'
pasta_painel = caminho_base.parent / '05_PAINEL'
pasta_historico_planos = caminho_base.parent / '04_HISTORICO_PLANOS'

print("✅ Mapeamento de pastas concluído com sucesso!")

✅ Mapeamento de pastas concluído com sucesso!


In [2]:
# 📥 Importando e preparando arquivos para desagregação

# Ler arquivos em parquet
df_forecast_vendas_krona_PRODUTO = pd.read_parquet(pasta_staging_parquet / 'df_forecast_vendas_krona_PRODUTO.parquet')
df_forecast_vendas_krona_CLIENTE = pd.read_parquet(pasta_staging_parquet / 'df_forecast_vendas_krona_CLIENTE.parquet')

# Ler arquivos CSV de planos de demanda
df_plano_consenso_regional = pd.read_csv(
    pasta_historico_planos / 'PLANO_REGIONAL.csv',
    sep=';',              # separador de colunas
    decimal=',',          # vírgula como separador decimal
    thousands='.',        # ponto como separador de milhar
    engine='python'
)

df_plano_consenso_cliente = pd.read_csv(
    pasta_historico_planos / 'PLANO_CLIENTE.csv',
    sep=';',               # separador de colunas
    decimal=',',           # vírgula como separador decimal
    thousands='.',         # ponto como separador de milhar
    engine='python',
    dtype={'COD_GRP_CLIENTE': str}   # força essa coluna como string
)

# Padronizar PERIODO como datetime
df_plano_consenso_regional['PERIODO'] = pd.to_datetime(df_plano_consenso_regional['PERIODO'])
df_plano_consenso_cliente['PERIODO'] = pd.to_datetime(df_plano_consenso_cliente['PERIODO'])

# Agrupar dados df_plano_consenso_regional somando os valores de DEMANDA_PLANEJADA
colunas_agrupamento = ['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA', 'PERIODO', 'VERSAO_PLANO']
df_plano_consenso_regional_grouped = df_plano_consenso_regional.groupby(colunas_agrupamento)['VALOR'].sum().reset_index()

# Agrupar dados df_plano_consenso_cliente somando os valores de DEMANDA_PLANEJADA
colunas_agrupamento = ['COD_GRP_CLIENTE', 'REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA', 'PERIODO', 'VERSAO_PLANO']
df_plano_consenso_cliente_grouped = df_plano_consenso_cliente.groupby(colunas_agrupamento)['VALOR'].sum().reset_index()

# Guardar variavel versao do plano da coluna VERSAO_PLANO, retornando um valor da coluna VERSAO_PLANO do df_plano_consenso_regional
versao_plano =  df_plano_consenso_regional_grouped['VERSAO_PLANO'].iloc[0]

print("✅ Arquivos importados e preparados com sucesso!")

✅ Arquivos importados e preparados com sucesso!


In [3]:
# Filtrar  df_plano_consenso_regional_grouped conform abaixo
df_forecast_vendas_krona_PRODUTO = df_forecast_vendas_krona_PRODUTO[
(df_forecast_vendas_krona_PRODUTO['REGIONAL_GESTOR'] == 'CO NO') &
(df_forecast_vendas_krona_PRODUTO['REGIONAL'] == 'CENTRO OESTE') &
(df_forecast_vendas_krona_PRODUTO['FAMILIA'] == '03 - ACESSORIOS KRONA') &
(df_forecast_vendas_krona_PRODUTO['PERIODO'] == '2026-03-01')
]

df_forecast_vendas_krona_PRODUTO


,EMPRESA,COD_PROD,DESC_PRODUTO,FAMILIA,LINHA,REGIONAL,REGIONAL_GESTOR,PERIODO,VOL_PREV
9488,KRONA - AP GOIÂNIA,0680,ASSENTO SANITARIO PREMIER BRANCO,03 - ACESSORIOS KRONA,62 - ASSENTOS SANITARIOS,CENTRO OESTE,CO NO,2026-03-01,766.11
9512,KRONA - AP GOIÂNIA,0681,ASSENTO SANITARIO PREMIER CARAMELO,03 - ACESSORIOS KRONA,62 - ASSENTOS SANITARIOS,CENTRO OESTE,CO NO,2026-03-01,54.83
9528,KRONA - AP GOIÂNIA,0682,ASSENTO SANITARIO PREMIER CINZA CLARO,03 - ACESSORIOS KRONA,62 - ASSENTOS SANITARIOS,CENTRO OESTE,CO NO,2026-03-01,47.54
9544,KRONA - AP GOIÂNIA,0683,ASSENTO SANITARIO PREMIER BEGE,03 - ACESSORIOS KRONA,62 - ASSENTOS SANITARIOS,CENTRO OESTE,CO NO,2026-03-01,12.48
9820,KRONA - AP GOIÂNIA,0727,TORNEIRA BOIA PARA CAIXA D AGUA 1/2 E 3/4,03 - ACESSORIOS KRONA,72 - TORNEIRAS BOIA,CENTRO OESTE,CO NO,2026-03-01,326.62
...,...,...,...,...,...,...,...,...,...
83416,KRONA - NORDESTE,0815,T1-TAMPA P/VALVULA PIA E LAVAT.,03 - ACESSORIOS KRONA,35 - VALVULAS P/ PIA E LAVATORIO,CENTRO OESTE,CO NO,2026-03-01,3.77
83524,KRONA - NORDESTE,0822,LAVATORIO KRONA - BRANCO 36 X 26 CM,03 - ACESSORIOS KRONA,68 - LAVATORIOS,CENTRO OESTE,CO NO,2026-03-01,21.80
83792,KRONA - NORDESTE,0861,MULTISIFAO DUPLO (TUBO EXTENSIVEL),03 - ACESSORIOS KRONA,37 - SIFOES,CENTRO OESTE,CO NO,2026-03-01,102.31
83836,KRONA - NORDESTE,0862,MULTISIFAO SIMPLES (TUBO EXTENSIVEL) COM 48,03 - ACESSORIOS KRONA,37 - SIFOES,CENTRO OESTE,CO NO,2026-03-01,276.85


In [57]:
# 📥 Desagregação dos plano REGIONAL

# Calculando a participação da cada linha de produto no volume total da combinação de chaves - REGIONAL_GESTOR, REGIONAL, FAMILIA, PERIODO
df_volume_desag_regional = df_forecast_vendas_krona_PRODUTO.copy()

chaves = ['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA', 'PERIODO']

# total por combinação
df_volume_desag_regional['TOTAL'] = df_volume_desag_regional.groupby(chaves)['VOL_PREV'].transform('sum')

# participação da linha dentro da combinação
df_volume_desag_regional['PARTIC'] = np.where(
    df_volume_desag_regional['TOTAL'] > 0,
    df_volume_desag_regional['VOL_PREV'] / df_volume_desag_regional['TOTAL'],
    0
)

# Mesclar participação com o plano consenso regional unificado
df_volume_desag_regional = pd.merge(
    df_volume_desag_regional,
    df_plano_consenso_regional_grouped,
    on=chaves,
    how='left'
)

# Renomear colunas VALOR E VOL_PREV para evitar conflitos
df_volume_desag_regional.rename(columns={'VOL_PREV': 'VOL_ESTATISTICO'}, inplace=True)
df_volume_desag_regional.rename(columns={'VALOR': 'VOL_CONSENSO'}, inplace=True)

# Calcular desagregação
df_volume_desag_regional['VOL_CONSENSO_DESAGREGADO'] = df_volume_desag_regional['PARTIC'] * df_volume_desag_regional['VOL_CONSENSO']

# # Excluir colunas TOTAL e PARTIC
# df_volume_desag_regional.drop(columns=['TOTAL', 'PARTIC'], inplace=True)

In [58]:
df_volume_desag_regional

,EMPRESA,COD_PROD,DESC_PRODUTO,FAMILIA,LINHA,REGIONAL,REGIONAL_GESTOR,PERIODO,VOL_ESTATISTICO,TOTAL,PARTIC,VERSAO_PLANO,VOL_CONSENSO,VOL_CONSENSO_DESAGREGADO
0,KRONA - AP GOIÂNIA,1061,JOGO ESGUICHO P/MANG.1/2 - 3/4 AMARELO/PRETO,03 - ACESSORIOS KRONA,16 - ESGUICHOS,CENTRO OESTE,CO NO,2026-03-01,44.95,182.53,0.25,20260204_101838,31877.32,7850.49
1,KRONA - AP GOIÂNIA,1061,JOGO ESGUICHO P/MANG.1/2 - 3/4 AMARELO/PRETO,03 - ACESSORIOS KRONA,16 - ESGUICHOS,CENTRO OESTE,CO NO,2026-04-01,39.57,163.71,0.24,20260204_101838,29262.36,7073.69
2,KRONA - AP GOIÂNIA,1061,JOGO ESGUICHO P/MANG.1/2 - 3/4 AMARELO/PRETO,03 - ACESSORIOS KRONA,16 - ESGUICHOS,CENTRO OESTE,CO NO,2026-05-01,48.19,194.21,0.25,20260204_101838,31696.20,7865.41
3,KRONA - AP GOIÂNIA,1061,JOGO ESGUICHO P/MANG.1/2 - 3/4 AMARELO/PRETO,03 - ACESSORIOS KRONA,16 - ESGUICHOS,CENTRO OESTE,CO NO,2026-06-01,62.26,246.17,0.25,20260204_101838,34510.99,8728.90
4,KRONA - AP GOIÂNIA,1061,JOGO ESGUICHO P/MANG.1/2 - 3/4 AMARELO/PRETO,03 - ACESSORIOS KRONA,16 - ESGUICHOS,NORTE,CO NO,2026-03-01,22.57,149.46,0.15,20260204_101838,15902.43,2401.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28123,KRONA - NORDESTE,2200118,"ASPERSOR 3/4 SUPERJET 2,5 X 1,8 - EMBALADO",AP - ASPERSORES,YC - ASPERSOR SUPERJET,IRRIGACAO 1,IRRIGACAO,2026-06-01,1.52,9.01,0.17,NaN,NaN,NaN
28124,KRONA - NORDESTE,2200118,"ASPERSOR 3/4 SUPERJET 2,5 X 1,8 - EMBALADO",AP - ASPERSORES,YC - ASPERSOR SUPERJET,IRRIGACAO 2,IRRIGACAO,2026-03-01,7.60,7.60,1.00,NaN,NaN,NaN
28125,KRONA - NORDESTE,2200118,"ASPERSOR 3/4 SUPERJET 2,5 X 1,8 - EMBALADO",AP - ASPERSORES,YC - ASPERSOR SUPERJET,IRRIGACAO 2,IRRIGACAO,2026-04-01,5.91,5.91,1.00,NaN,NaN,NaN
28126,KRONA - NORDESTE,2200118,"ASPERSOR 3/4 SUPERJET 2,5 X 1,8 - EMBALADO",AP - ASPERSORES,YC - ASPERSOR SUPERJET,IRRIGACAO 2,IRRIGACAO,2026-05-01,6.93,6.93,1.00,NaN,NaN,NaN


In [96]:
# 📥 Desagregação dos plano CLIENTE

# Calculando a participação da cada linha de produto no volume total da combinação de chaves - REGIONAL_GESTOR, REGIONAL, FAMILIA, PERIODO
df_volume_desag_cliente = df_forecast_vendas_krona_CLIENTE.copy()

# Renomear coluna COD_GRP_CLIENTE para COD_GRUPO_CLIENTE para manter padrão
df_volume_desag_cliente.rename(columns={'COD_GRUPO_CLIENTE': 'COD_GRP_CLIENTE'}, inplace=True)

chaves = ['COD_GRP_CLIENTE', 'REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA', 'PERIODO']

# total por combinação
df_volume_desag_cliente['TOTAL'] = df_volume_desag_cliente.groupby(chaves)['VOL_PREV'].transform('sum')

# participação da linha dentro da combinação
df_volume_desag_cliente['PARTIC'] = np.where(
    df_volume_desag_cliente['TOTAL'] > 0,
    df_volume_desag_cliente['VOL_PREV'] / df_volume_desag_cliente['TOTAL'],
    0
)

# Mesclar participação com o plano consenso regional unificado
df_volume_desag_cliente = pd.merge(
    df_volume_desag_cliente,
    df_plano_consenso_cliente_grouped,
    on=chaves,
    how='left'
)

# Renomear colunas VALOR E VOL_PREV para evitar conflitos
df_volume_desag_cliente.rename(columns={'VOL_PREV': 'VOL_ESTATISTICO'}, inplace=True)
df_volume_desag_cliente.rename(columns={'VALOR': 'VOL_CONSENSO'}, inplace=True)

# Calcular desagregação
df_volume_desag_cliente['VOL_CONSENSO'] = df_volume_desag_cliente['PARTIC'] * df_volume_desag_cliente['VOL_CONSENSO']

# Excluir colunas TOTAL e PARTIC
df_volume_desag_cliente.drop(columns=['TOTAL', 'PARTIC'], inplace=True)

print("✅ Desagregação concluída com sucesso!")

✅ Desagregação concluída com sucesso!


In [101]:
# 📥 Unificação de dados CLIENTE E REGIONAL

# Agrupar dados df_volume_desag_cliente
colunas_agrupar = ['EMPRESA', 'COD_PROD', 'DESC_PRODUTO', 'FAMILIA', 'LINHA', 'REGIONAL', 'REGIONAL_GESTOR', 'PERIODO']

df_volume_desag_cliente_agrupado = df_volume_desag_cliente.groupby(colunas_agrupar).agg({
    'VOL_ESTATISTICO': 'sum',
    'VOL_CONSENSO': 'sum'
}).reset_index()

# Se soma de VOL_CONSENS de df_volume_desag_cliente_agrupado for maior que 0
if df_volume_desag_cliente_agrupado['VOL_CONSENSO'].sum() > 0:
    # Unificar dataframes de df_volume_desag_regional e df_volume_desag_cliente_agrupado
    df_plano_final_krona = pd.concat([df_volume_desag_regional, df_volume_desag_cliente_agrupado], ignore_index=True)
else:
    df_plano_final_krona = df_volume_desag_regional.copy()

# Agrupar dados df_plano_final_krona somando os valores de VOL_ESTATISTICO e VOL_CONSENSO
colunas_agrupamento_final = ['EMPRESA', 'COD_PROD', 'DESC_PRODUTO', 'FAMILIA', 'LINHA', 'REGIONAL', 'REGIONAL_GESTOR', 'PERIODO']
df_plano_final_krona = df_plano_final_krona.groupby(colunas_agrupamento_final).agg({
    'VOL_ESTATISTICO': 'sum',
    'VOL_CONSENSO': 'sum'
}).reset_index()

# Carregar parquet Dim_Produtos_Vendas_krona para buscar peso unitário
df_dim_peso_unit_vendas = pd.read_parquet(pasta_staging_parquet / 'df_dim_peso_unit_vendas.parquet')

# Mesclar peso unitário com o plano final
df_plano_final_krona = pd.merge(
    df_plano_final_krona,
    df_dim_peso_unit_vendas,
    on=['EMPRESA', 'COD_PROD'],
    how='left'
)

df_plano_final_krona['QTD_CONSENSO'] = df_plano_final_krona['VOL_CONSENSO'] / df_plano_final_krona['PESO_UNIT']
df_plano_final_krona['QTD_ESTATISTICO'] = df_plano_final_krona['VOL_ESTATISTICO'] / df_plano_final_krona['PESO_UNIT']

# Adicionar coluna de versão do plano
df_plano_final_krona['VERSAO_PLANO'] = versao_plano

# Carregar parquet com Demanda de Lançamento
df_produtos_lancamento = pd.read_parquet(pasta_staging_parquet / 'DEMANDA_LANCAMENTO_PRODUTOS_KRONA.parquet')

# Renomear colunas para facilitar merge
df_produtos_lancamento.rename(columns={'CD': 'EMPRESA'}, inplace=True)

# Filtrar colunda EMPRESA de df_produtos_lancamento que contenha KRONA
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['EMPRESA'].str.contains('KRONA')]

# Filtrar PERIODO da df_produtos_lancamento pelas datas que constam na df_plano_final_krona
periodos_plano = df_plano_final_krona['PERIODO'].unique()
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['PERIODO'].isin(periodos_plano)].reset_index(drop=True)


# Expandir produtos de lançamento para cobrir todos os períodos do plano
def expandir_produtos_lancamento(df_produtos_lancamento, periodos_plano):
    """
    Expande o dataframe de lançamentos para cobrir todos os períodos do plano.
    Se faltar período, duplica a demanda do último mês disponível.
    Adiciona coluna STATUS para diferenciar demanda existente e criada.
    """
    def expandir_grupo(grupo):
        grupo = grupo.sort_values('PERIODO')
        ultimo_valor = grupo['QTD'].iloc[-1]
        existentes = grupo['PERIODO'].unique()
        faltantes = [p for p in periodos_plano if p not in existentes]

        grupo['STATUS'] = 'DEMANDA_EXISTENTE'

        if faltantes:
            novos = pd.DataFrame({
                'COD_PROD': grupo['COD_PROD'].iloc[0],
                'EMPRESA': grupo['EMPRESA'].iloc[0],
                'PERIODO': faltantes,
                'QTD': ultimo_valor,
                'STATUS': 'DEMANDA_CRIADA'
            })
            grupo = pd.concat([grupo, novos], ignore_index=True)

        return grupo

    # 🚀 iterando manualmente pelos grupos (sem apply → sem warning)
    grupos_expandidos = []
    for _, grupo in df_produtos_lancamento.groupby(['COD_PROD','EMPRESA']):
        grupos_expandidos.append(expandir_grupo(grupo))

    df_expandido = pd.concat(grupos_expandidos, ignore_index=True)
    df_expandido = df_expandido.sort_values(['COD_PROD','EMPRESA','PERIODO'])

    return df_expandido

df_lancamentos_expandido = expandir_produtos_lancamento(df_produtos_lancamento, periodos_plano)

# Salvar df_lancamentos_expandido em excel
df_lancamentos_expandido.to_excel(pasta_historico_planos / f'PLANO_LANCAMENTOS_EXPANDIDO.xlsx', index=False)

# Criar colunas no df_lancamentos_expandido que faltam para unir com o df_plano_final_krona
df_lancamentos_expandido['DESC_PRODUTO'] = ''
df_lancamentos_expandido['FAMILIA'] = ''
df_lancamentos_expandido['LINHA'] = ''
df_lancamentos_expandido['REGIONAL'] = ''
df_lancamentos_expandido['REGIONAL_GESTOR'] = ''
df_lancamentos_expandido['VOL_ESTATISTICO'] = 0
df_lancamentos_expandido['VOL_CONSENSO'] = 0
df_lancamentos_expandido['PESO_UNIT'] = 0
df_lancamentos_expandido['QTD_ESTATISTICO'] = 0
df_lancamentos_expandido['VERSAO_PLANO'] = versao_plano

# Renomear coluna QTD para QTD_CONSENSO
df_lancamentos_expandido.rename(columns={'QTD': 'QTD_CONSENSO'}, inplace=True)

# Ordenar colunas do df_lancamentos_expandido para ficar igual ao df_plano_final_krona
colunas_ordem = df_plano_final_krona.columns.tolist()
df_lancamentos_expandido = df_lancamentos_expandido[colunas_ordem]

# Unir df_plano_final_krona com df_lancamentos_expandido
df_plano_final_krona = pd.concat([df_plano_final_krona, df_lancamentos_expandido], ignore_index=True)

# Eliminar linhas com QTD_ESTATISTICO igual a zero
df_plano_final_krona = df_plano_final_krona[df_plano_final_krona['QTD_ESTATISTICO'] > 0].reset_index(drop=True)

# Salvar em um banco de dados em Parquet, adicionando a versão do plano na nomenclatura do arquivo
df_plano_final_krona.to_parquet(pasta_historico_planos / "BD_PLANOS" / f'PLANO_CONSENSO_KRONA_{versao_plano}.parquet', index=False)

# Agrupar dados e gerar plano para calculo de produção
df_demanda_plano_producao = df_plano_final_krona.groupby(['COD_PROD', 'EMPRESA', 'PERIODO']).agg({
    'QTD_CONSENSO': 'sum'
}).reset_index()

# Salvar plano de demanda em Excel, gerando um arquivo por EMPRESA, excluir coluna EMPRESA e renomear as colunas antes de enviar
for empresa, grupo in df_demanda_plano_producao.groupby('EMPRESA'):
    grupo['Alm.'] = 1
    grupo = grupo.drop(columns=['EMPRESA']).rename(columns={
        'COD_PROD': 'Cod',
        'PERIODO': 'Dt.',
        'QTD_CONSENSO': 'Qtde.'
    })
    grupo = grupo[['Cod', 'Alm.', 'Qtde.', 'Dt.']]
    grupo['Dt.'] = grupo['Dt.'].dt.date

    grupo.to_csv(
        pasta_historico_planos / 'PLANOS_PCP' / f'DEMANDA_LTP_{empresa}_{versao_plano}.csv',
        index=False,
        sep=';',
        decimal=',',
        float_format='%.3f'
    )

    # grupo.to_excel(pasta_historico_planos / 'PLANOS_PCP' / f'DEMANDA_LTP_{empresa}_{versao_plano}.xlsx', index=False)

print("✅ Volumes gerados, bases geradas, plano para produção gerado com sucesso!")

✅ Volumes gerados, bases geradas, plano para produção gerado com sucesso!


✅ Volumes gerados, bases geradas, plano para produção gerado com sucesso!


In [99]:
timer.finalizar()
print("🎯 Processo concluído com sucesso!")


⏱️ Tempo total de processamento: 0 min 0.8 s
🎯 Processo concluído com sucesso!
